## Prepare Data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [1]:
from pathlib import Path
import os

In [2]:
#NOTE: DONT USE RELATIVE PATHS FOR THE MODELS PROVIDED BY L5
experiments_directory = Path(Path(os.path.abspath('')).parent.parent, "Experiments")
experiments_directory.mkdir(parents=True, exist_ok=True)

data_directory = Path(experiments_directory, "data")
data_directory.mkdir(parents=True, exist_ok=True)

prediction_directory = Path(experiments_directory, "prediction")
prediction_directory.mkdir(parents=True, exist_ok=True)

save_directory = Path(prediction_directory, "saved_outputs")
save_directory.mkdir(parents=True, exist_ok=True)

In [3]:
import os
os.chdir(prediction_directory)

In [4]:
%%writefile requirements.txt
l5kit
pyyaml
ray==2.0.0rc1
ray[air]
wandb
optuna

Overwriting requirements.txt


In [5]:
%%capture
# !pip install -r requirements.txt
!pip install l5kit pyyaml wandb
!pip install ray==2.0.0rc1
!pip install "ray[air]"
!pip install optuna

In [6]:
import wandb
wandb.login()

wandb: Currently logged in as: a-sh0ts. Use `wandb login --relogin` to force relogin


True

In [7]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import os

In [8]:
# Run information
wandb_entity = "l5-demo"
project_name = "l5-prediction"
run_name = "download-l5-data"
run_type = "download"
run_description = """
Download data for the task of training a prediction model
"""
tags = ["download", "data"]

In [9]:
run = wandb.init(
    entity=wandb_entity,
    project=project_name,
    job_type=run_type,
    name=run_name,
    notes=run_description,
    tags=tags
)

wandb: Currently logged in as: a-sh0ts (l5-demo). Use `wandb login --relogin` to force relogin


In [10]:
artifact_entity = "l5-demo"
artifact_project = "l5-common"
artifact_name = "l5-data"
artifact_alias = "latest"
artifact_type = "dataset"

In [11]:
artifact = run.use_artifact(f"{artifact_entity}/{artifact_project}/{artifact_name}:{artifact_alias}", type=artifact_type)

In [12]:
_ = artifact.download(data_directory)

wandb: Downloading large artifact l5-data:latest, 2386.92MB. 517 files... Done. 0:0:0.1


In [13]:
#BUG: need to seperate runs into download and training due to issues with routing runs after ray.tune
run.finish()

In [14]:
# Dataset is assumed to be on the folder specified
# in the L5KIT_DATA_FOLDER environment variable

# get config
cfg = load_config_data(Path(data_directory, "configurations", "agent_motion_config.yaml"))
l5_data_location = Path(data_directory, "dataset")
# run.config.update(cfg)

In [15]:
# cfg["zarr_dataset_location"] = l5_data_location
os.environ["L5KIT_DATA_FOLDER"] = str(l5_data_location)

## Model

Our baseline is a simple `resnet50` pretrained on `imagenet`. We must replace the input and the final layer to address our requirements.

In [16]:
def build_model(cfg: Dict) -> torch.nn.Module:
    # load pre-trained Conv2D model
    model = resnet50(pretrained=True)

    # change input channels number to match the rasterizer's output
    num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
    num_in_channels = 3 + num_history_channels
    model.conv1 = nn.Conv2d(
        num_in_channels,
        model.conv1.out_channels,
        kernel_size=model.conv1.kernel_size,
        stride=model.conv1.stride,
        padding=model.conv1.padding,
        bias=False,
    )
    # change output size to (X, Y) * number of future states
    num_targets = 2 * cfg["model_params"]["future_num_frames"]
    model.fc = nn.Linear(in_features=2048, out_features=num_targets)

    return model

In [17]:
def forward(data, model, criterion):
    inputs = data["image"]
    target_availabilities = data["target_availabilities"].unsqueeze(-1)
    targets = data["target_positions"]
    # Forward pass
    outputs = model(inputs).reshape(targets.shape)
    loss = criterion(outputs, targets)
    # not all the output steps are valid, but we can filter them out from the loss using availabilities
    loss = loss * target_availabilities
    loss = loss.mean()
    return loss, outputs

In [18]:
def train_prediction_model_epoch(data, model, criterion, optimizer):
    loss, outputs = forward(data, model, criterion)
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, outputs

## Load the Train Data

Our data pipeline map a raw `.zarr` folder into a multi-processing instance ready for training by:
- loading the `zarr` into a `ChunkedDataset` object. This object has a reference to the different arrays into the zarr (e.g. agents and traffic lights);
- wrapping the `ChunkedDataset` into an `AgentDataset`, which inherits from torch `Dataset` class;
- passing the `AgentDataset` into a torch `DataLoader`

In [19]:
import ray.train as train
from ray.air import session, Checkpoint

In [20]:
from ray import tune
from ray.tune.tuner import Tuner

In [21]:
def train_prediction_model(tuner_cfg : Dict):
    # ==== INIT DATASET
    #TODO fix this data flow if it doesnt make sense
    dm = LocalDataManager()
    
    shuffle = tuner_cfg["shuffle"]
    batch_size = int(tuner_cfg["batch_size"])
    num_workers = tuner_cfg["num_workers"]
    lr = tuner_cfg["lr"]
    max_num_steps = int(tuner_cfg["max_num_steps"])
    dataset_key = tuner_cfg["dataset_key"]
    cfg = tuner_cfg["cfg"]
    
    rasterizer = build_rasterizer(cfg, dm)

    train_zarr = ChunkedDataset(dm.require(dataset_key)).open()
    train_dataset = AgentDataset(cfg, train_zarr, rasterizer)

    batch_size_per_worker = batch_size // session.get_world_size()
    train_dataloader = DataLoader(train_dataset, shuffle=shuffle, batch_size=batch_size_per_worker, num_workers=num_workers)
    train_dataloader = train.torch.prepare_data_loader(train_dataloader)
    
    # ==== INIT MODEL
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = build_model(cfg)
    model = train.torch.prepare_model(model)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss(reduction="none")

    # ==== TRAIN LOOP
    tr_it = iter(train_dataloader)
    # progress_bar = tqdm(range(cfg["train_params"].get("max_num_steps", 5)))
    progress_bar = range(max_num_steps)
    num_checkpoints = 5
    steps_before_checkpointing = max_num_steps // num_checkpoints
    losses_train = []
    for step in progress_bar:
        try:
            data = next(tr_it)
        except StopIteration:
            tr_it = iter(train_dataloader)
            data = next(tr_it)
            
        model.train()
        torch.set_grad_enabled(True)
        loss, _ = train_prediction_model_epoch(data, model, criterion, optimizer)
        losses_train.append(loss.item())
        avg_loss = np.mean(losses_train)
        metrics = {
            "loss": loss.item(),
            "avg_loss": avg_loss
        }
        
        #NOTE: To prevent stdout bloat
        if train.world_rank() == 0:
            print(metrics)
        # progress_bar.set_description(f"loss: {metrics["loss"]} loss(avg): {metrics["loss(avg)"]}")
        
        #TODO: fix this lmao brain tired
        if steps_before_checkpointing > 0:
            if step%steps_before_checkpointing==0:
                session.report(
                    metrics=metrics,
                    checkpoint=Checkpoint.from_dict(dict(step=step, model=model)),
                )
            elif step==max_num_steps-1:
                session.report(
                    metrics=metrics,
                    checkpoint=Checkpoint.from_dict(dict(step=step, model=model)),
                )
            else:
                session.report(
                    metrics=metrics
                )
        else:
            session.report(
                metrics=metrics
            )

# Training

note: if you're on MacOS and using `py_satellite` rasterizer, you may need to disable opencv multiprocessing by adding:
`cv2.setNumThreads(0)` before the following cell. This seems to only affect running in python notebook and it's caused by the `cv2.warpaffine` function

In [22]:
from ray.train.torch import TorchTrainer
from ray.air.config import RunConfig, ScalingConfig
from ray.air.callbacks.wandb import WandbLoggerCallback

In [23]:
trainer = TorchTrainer(
    train_loop_per_worker=train_prediction_model,
    scaling_config=ScalingConfig(num_workers=3, use_gpu=False), #TODO: Add logic to check if GPU is available here
)

2022-08-19 15:45:55,144	INFO worker.py:1487 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265.


In [24]:
tuner_train_config = {}
##static
tuner_train_config["shuffle"] = cfg["train_data_loader"]["shuffle"]
#TODO: what is this useful for?
tuner_train_config["num_workers"] = cfg["train_data_loader"]["num_workers"]
tuner_train_config["dataset_key"] = cfg["train_data_loader"]["key"]

##tunable
# tuner_train_config["max_num_steps"] = tune.quniform(50, 500, 50)
tuner_train_config["max_num_steps"] = tune.quniform(5, 50, 5)
tuner_train_config["lr"] = tune.loguniform(1e-3, 1e-2)
tuner_train_config["batch_size"] = tune.quniform(6, 24, 3)

tuner_train_config["cfg"] = cfg

In [25]:
from ray.tune.logger import LoggerCallback
from typing import Dict, List

In [26]:
# ## For now passing the current run context to this callback to better organize the models logged
# class TrialEndModelSaveCallback(LoggerCallback):
    
#     def __init__(self, run, save_directory):
#         self.run = run
#         self.save_directory = save_directory
    
#     def on_trial_complete(self, iteration: int, trials: List["Trial"],
#                           trial: "Trial", **info):
#         print("here")
#         trial_name = trial._trainable_name(include_trial_id=True)
#         print(trial_name)
#         final_model_checkpoint = trial.checkpoint.to_air_checkpoint().to_dict()
#         print(final_model_checkpoint)
#         model = final_model_checkpoint["model"]
#         print(model)
#         final_model = torch.jit.script(model.cpu())
#         path_to_save = f"{trial_name}-trained_model"
#         final_model.save(path_to_save)

In [27]:
from ray.tune.stopper import ExperimentPlateauStopper
from ray.tune.search.optuna import OptunaSearch

In [28]:
n_search_attempts = 10

In [29]:
optuna_search = OptunaSearch()

In [30]:
tuner = Tuner(
        trainer,
        tune_config=tune.TuneConfig(
            metric="avg_loss", #loss or avg_loss here?
            mode="min",
            search_alg=optuna_search,
            num_samples=n_search_attempts,
        ),
        param_space={
            "train_loop_config": tuner_train_config
        },
        run_config=RunConfig(
            stop=ExperimentPlateauStopper("avg_loss"),
            callbacks=[WandbLoggerCallback(project=f"{project_name}-trials", save_checkpoints=True), 
                                        # TrialEndModelSaveCallback(run, save_directory)
                                       ]
                            )
    )

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Executing `.fit()` may leave less than 20% of CPUs in this cluster for Dataset execution, which can lead to resource contention or hangs. To avoid this, reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer scaling_config. See https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune for more info.
  


In [31]:
analysis = tuner.fit()

(RayTrainWorker pid=16406) {'loss': 80.25357055664062, 'avg_loss': 71.58842504435572}
Result for TorchTrainer_5c2b750a:
  _time_this_iter_s: 7.779364585876465
  _timestamp: 1660924500
  _training_iteration: 29
  avg_loss: 71.58842504435572
  date: 2022-08-19_15-55-01
  done: false
  experiment_id: eae92d0a63fc4567a5392ff8a089dc67
  hostname: anish-l5-kit
  iterations_since_restore: 29
  loss: 80.25357055664062
  node_ip: 10.150.0.3
  pid: 16328
  time_since_restore: 248.99855041503906
  time_this_iter_s: 7.430035829544067
  time_total_s: 248.99855041503906
  timestamp: 1660924501
  timesteps_since_restore: 0
  training_iteration: 29
  trial_id: 5c2b750a
  warmup_time: 0.005020856857299805
  
Result for TorchTrainer_5c2b750a:
  _time_this_iter_s: 7.822077035903931
  _timestamp: 1660924508
  _training_iteration: 30
  avg_loss: 71.98505894343059
  date: 2022-08-19_15-55-08
  done: false
  experiment_id: eae92d0a63fc4567a5392ff8a089dc67
  hostname: anish-l5-kit
  iterations_since_restore: 

wandb: Adding directory to artifact (/home/jupyter/ray_results/TorchTrainer_2022-08-19_15-45-58/TorchTrainer_5c2b750a_3_batch_size=24.0000,format_version=4,future_num_frames=50,history_num_frames=0,model_architecture=resnet50,_2022-08-19_15-50-48/checkpoint_000004)... 

Result for TorchTrainer_5c2b750a:
  _time_this_iter_s: 8.145943641662598
  _timestamp: 1660924560
  _training_iteration: 37
  avg_loss: 68.69699362161997
  date: 2022-08-19_15-56-00
  done: false
  experiment_id: eae92d0a63fc4567a5392ff8a089dc67
  hostname: anish-l5-kit
  iterations_since_restore: 37
  loss: 67.09259033203125
  node_ip: 10.150.0.3
  pid: 16328
  should_checkpoint: true
  time_since_restore: 308.62834787368774
  time_this_iter_s: 8.510472297668457
  time_total_s: 308.62834787368774
  timestamp: 1660924560
  timesteps_since_restore: 0
  training_iteration: 37
  trial_id: 5c2b750a
  warmup_time: 0.005020856857299805
  


Done. 1.0s


(RayTrainWorker pid=16406) {'loss': 36.63917541503906, 'avg_loss': 67.85336682670994}
Result for TorchTrainer_5c2b750a:
  _time_this_iter_s: 8.376948118209839
  _timestamp: 1660924568
  _training_iteration: 38
  avg_loss: 67.85336682670994
  date: 2022-08-19_15-56-08
  done: false
  experiment_id: eae92d0a63fc4567a5392ff8a089dc67
  hostname: anish-l5-kit
  iterations_since_restore: 38
  loss: 36.63917541503906
  node_ip: 10.150.0.3
  pid: 16328
  time_since_restore: 316.6358058452606
  time_this_iter_s: 8.007457971572876
  time_total_s: 316.6358058452606
  timestamp: 1660924568
  timesteps_since_restore: 0
  training_iteration: 38
  trial_id: 5c2b750a
  warmup_time: 0.005020856857299805
  
Result for TorchTrainer_5c2b750a:
  _time_this_iter_s: 7.18516206741333
  _timestamp: 1660924575
  _training_iteration: 39
  avg_loss: 67.7737532151051
  date: 2022-08-19_15-56-15
  done: false
  experiment_id: eae92d0a63fc4567a5392ff8a089dc67
  hostname: anish-l5-kit
  iterations_since_restore: 39
 

...

Result for TorchTrainer_c9513f7c:
  _time_this_iter_s: 8.360788345336914
  _timestamp: 1660925788
  _training_iteration: 10
  avg_loss: 83.46885242462159
  date: 2022-08-19_16-16-29
  done: false
  experiment_id: 33253c1dec9943b3abbfa53eda24348c
  hostname: anish-l5-kit
  iterations_since_restore: 10
  loss: 36.51075744628906
  node_ip: 10.150.0.3
  pid: 25888
  should_checkpoint: true
  time_since_restore: 114.68820333480835
  time_this_iter_s: 8.460174083709717
  time_total_s: 114.68820333480835
  timestamp: 1660925789
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: c9513f7c
  warmup_time: 0.005364418029785156
  


Done. 1.0s
wandb: Adding directory to artifact (/home/jupyter/ray_results/TorchTrainer_2022-08-19_15-45-58/TorchTrainer_c9513f7c_10_batch_size=24.0000,format_version=4,future_num_frames=50,history_num_frames=0,model_architecture=resnet50_2022-08-19_16-14-31/checkpoint_000005)... Done. 0.3s


Result for TorchTrainer_c9513f7c:
  _time_this_iter_s: 8.360788345336914
  _timestamp: 1660925788
  _training_iteration: 10
  avg_loss: 83.46885242462159
  date: 2022-08-19_16-16-29
  done: true
  experiment_id: 33253c1dec9943b3abbfa53eda24348c
  experiment_tag: 10_batch_size=24.0000,format_version=4,future_num_frames=50,history_num_frames=0,model_architecture=resnet50,render_ego_history=True,step_time=0.1000,dataset_meta_key=meta_json,disable_traffic_light_faces=False,ego_center=0_25_0_5,filter_agents_threshold=0.5000,map_type=py_semantic,pixel_size=0_5_0_5,raster_size=224_224,satellite_map_key=aerial_map_aerial_map_png,semantic_map_key=semantic_map_semantic_map_pb,set_origin_to_bottom=True,batch_size=12,key=scenes_sample_zarr,num_workers=16,shuffle=True,checkpoint_every_n_steps=10000,eval_every_n_steps=10000,max_num_steps=5,batch_size=12,key=scenes_sample_zarr,num_workers=16,shuffle=False,dataset_key=scenes_sample_zarr,lr=0.0017,max_num_steps=10.0000,num_workers=16,shuffle=True
  hos

2022-08-19 16:16:39,717	INFO tune.py:759 -- Total run time: 1841.64 seconds (1841.45 seconds for the tuning loop).


avg_loss,▂▁▁▄▄▅▆█▇▇
iterations_since_restore,▁▂▃▃▄▅▆▆▇█
loss,▁▁▁▆▃▄▆█▁▂
time_since_restore,▁▂▃▃▄▅▆▆▇█
time_this_iter_s,█▁▁▁▁▁▁▁▁▁
time_total_s,▁▂▃▃▄▅▆▆▇█
timestamp,▁▂▃▃▄▅▆▆▇█
timesteps_since_restore,▁▁▁▁▁▁▁▁▁▁
training_iteration,▁▂▃▃▄▅▆▆▇█
warmup_time,▁▁▁▁▁▁▁▁▁▁
avg_loss,83.46885


In [34]:
import time

In [35]:
#does sleeping help???
time.sleep(30)

In [32]:
analysis_df = analysis.get_dataframe()

In [33]:
analysis_df

,loss,avg_loss,_timestamp,_time_this_iter_s,_training_iteration,time_this_iter_s,should_checkpoint,done,timesteps_total,episodes_total,...,config/train_loop_config/cfg/train_data_loader/num_workers,config/train_loop_config/cfg/train_data_loader/shuffle,config/train_loop_config/cfg/train_params/checkpoint_every_n_steps,config/train_loop_config/cfg/train_params/eval_every_n_steps,config/train_loop_config/cfg/train_params/max_num_steps,config/train_loop_config/cfg/val_data_loader/batch_size,config/train_loop_config/cfg/val_data_loader/key,config/train_loop_config/cfg/val_data_loader/num_workers,config/train_loop_config/cfg/val_data_loader/shuffle,logdir
0,41.008888,94.537867,1660924095,3.218059,35,3.654188,True,False,NaN,NaN,...,16,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-08...
1,109.761116,67.864799,1660924236,4.230319,25,4.711835,True,False,NaN,NaN,...,16,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-08...
2,41.149384,70.176504,1660924618,7.601075,45,8.216519,True,False,NaN,NaN,...,16,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-08...
3,94.941132,92.686515,1660924787,3.154223,40,3.595659,True,False,NaN,NaN,...,16,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-08...
4,106.942398,82.296616,1660924977,5.036870,30,5.481915,True,False,NaN,NaN,...,16,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-08...
5,24.701918,90.694370,1660925174,3.202257,50,3.554569,True,False,NaN,NaN,...,16,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-08...
6,40.301899,70.757032,1660925455,7.444295,30,7.902245,True,False,NaN,NaN,...,16,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-08...
7,75.253006,69.915300,1660925564,3.349678,20,3.707979,True,False,NaN,NaN,...,16,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-08...
8,240.057770,70.046607,1660925661,5.844266,10,5.798168,True,False,NaN,NaN,...,16,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-08...
9,36.510757,83.468852,1660925788,8.360788,10,8.460174,True,False,NaN,NaN,...,16,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-08...


In [36]:
# analysis.get_best_result().metrics
# best_result = 

In [37]:
# Run information
wandb_entity = "l5-demo"
project_name = "l5-prediction"
run_name = "train-prediction-model"
run_type = "train"
run_description = """
Train prediction model
"""
tags = ["train", "prediction"]

In [38]:
run = wandb.init(
    entity=wandb_entity,
    project=project_name,
    job_type=run_type,
    name=run_name,
    notes=run_description,
    tags=tags,
    config=cfg
)

In [ ]:
#BUG: to force a connection on the lineage graph
artifact = run.use_artifact(f"{artifact_entity}/{artifact_project}/{artifact_name}:{artifact_alias}", type=artifact_type)

In [39]:
analysis_table = wandb.Table(dataframe=analysis_df)

In [40]:
#BUG: run gets lost after tune job due to change in cwd. Forced to make 2 runs
if len(analysis_table.data) == 0:
    raise ValueError("bad table for some reason")
else:
    run.log({"analysis_table": analysis_table})
    run.finish()